In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
# Importando o módulo de Regressão Linear do scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

Lucro Líquido                             0.300
Fluxo de Caixa de Financiamentos (FCF)    0.254
Crescimento_Médio_Anual                   0.244
Cotação                                   0.238
2015                                      0.236
EV/EBITDA                                 0.220
ROA                                       0.219
P/VPA                                     0.198
PSR                                       0.176
ROE                                       0.174
Valor de Mercado                          0.123
Valor_de_Mercado                          0.123
Fluxo de Caixa de Investimentos (FCI)     0.118
Participação no Ibovespa                  0.094
Participação_no_Ibovespa                  0.094
Participação no IBRX 100                  0.094
Participação_no_IBRX_100                  0.094
Capex/FCO                                 0.054
Volume_Negociado                          0.053
Volume Negociado                          0.053
Lucro Operacional (EBIT)                

In [2]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código de Neg."),on="Código de Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed: 0",axis=1,inplace=True)

In [3]:
#Removando uma coluna de notas que nao foi utilizada, pois tem dados faltando
uniaoAbas.drop("NotaFundamentei20200201",axis=1,inplace=True)

In [4]:
#Selecionando todas as empresas que possuem notas e estão com seus dados completos
empresasComNotas=uniaoAbas.replace("-",np.nan)
empresasComNotas.dropna(inplace=True)

In [5]:
#Normalizando o dataFrames
#.drop('NotaFundamentei', axis=1)
dfCNota=normalizaDf(empresasComNotas)

In [6]:
#Gerando a analise de correlação dos atributos, já excluindo a "NotaFundamentei" da resposta
analise=analiseCorr("NotaFundamentei",dfCNota)[1:]

In [7]:
#Selecionando os melhores atributos para treinar o regressor
melhoresAtributos=[]
count=0
for k in analise.values:
    if (k > 0.3) | (k< -0.3):
        melhoresAtributos.append(analise.axes[0][count])
    count+=1

In [10]:
regression = LinearRegression()

LinearRegression()

In [292]:
notasPreditas=[]
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
for index in dfCNota.index.values:
    #Selecionando todas as empresas, exceto a que será usada como teste
    X_treino=dfCNota[dfCNota.index.values!=index][melhoresAtributos]
    #Selecionando todas as notas, exceto da empresa que será usada como teste
    y_treino=empresasComNotas[empresasComNotas.index.values!=index]["NotaFundamentei"]
    regression.fit(X_treino, y_treino)
    #Selecionando a empresa usada para testar
    X_test=dfCNota[dfCNota.index.values==index][melhoresAtributos]
    #Selecionando a nota da empresa usada para testar
    y_test=empresasComNotas[empresasComNotas.index.values==index]["NotaFundamentei"].array[0]
    #print(y_test)
    #Descobrindo a nota predita da empresa
    Y_pred=regression.predict(X_test)[0].round(1)
    #Adicionando a list que se tornará uma coluna
    notasPreditas.append(Y_pred)
    
notasF=empresasComNotas[empresasComNotas.index.values==dfCNota.index.values]["NotaFundamentei"]
print(f'MAE {mean_absolute_error(notasF, notasPreditas)}')
print(f'MSE {mean_squared_error(notasF, notasPreditas)}')
notasPreditas

MAE 1.434375
MSE 4.5234375


[-0.3,
 2.9,
 0.8,
 3.9,
 4.0,
 -3.2,
 4.4,
 3.4,
 3.2,
 2.9,
 3.1,
 3.8,
 4.8,
 4.4,
 3.4,
 3.8,
 3.3,
 6.2,
 2.4,
 3.2,
 2.7,
 2.7,
 3.9,
 3.0,
 2.2,
 4.4,
 4.6,
 4.5,
 4.8,
 8.7,
 4.8,
 3.1]

In [293]:
emp=empresasComNotas.loc[dfCNota.index.values,["Nome", "Código de Neg.","NotaFundamentei"]]

In [295]:
emp["NotaPredita"]=notasPreditas
emp["NotaFundamentei-NotaPredita"]=emp["NotaFundamentei"]-emp["NotaPredita"]

In [296]:
emp.sort_values("NotaFundamentei",ascending=False)

,Nome,Código de Neg.,NotaFundamentei,NotaPredita,NotaFundamentei-NotaPredita
385,Weg,WEGE3,5.0,4.8,0.2
228,Lojas Renner,LREN3,4.8,3.3,1.5
136,Engie Brasil,EGIE3,4.8,3.1,1.7
149,Fleury,FLRY3,4.6,3.8,0.8
31,Ambev S/A,ABEV3,4.6,-0.3,4.9
278,RaiaDrogasil,RADL3,4.5,3.0,1.5
39,B3,B3SA3,4.5,2.9,1.6
133,Energias BR,ENBR3,4.4,2.9,1.5
175,Hypera,HYPE3,4.3,4.4,-0.1
248,Multiplan,MULT3,4.1,3.2,0.9


In [297]:
emp.to_excel("Regressor de Dados.xlsx")